# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [52]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [53]:
# your code here
df = pd.read_csv('../Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [54]:
# your code here
df.columns
df.columns = [i.strip('. ').replace(' ','_') for i in df.columns]

Let's evaluate the dataset by looking at the `head` function.

In [55]:
# your code here
df.head()
df.columns

Index(['Serial_No', 'GRE_Score', 'TOEFL_Score', 'University_Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance_of_Admit'],
      dtype='object')

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [56]:
# your code here
df.isna().sum()

Serial_No            0
GRE_Score            0
TOEFL_Score          0
University_Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance_of_Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [57]:
# your code here
df.index = df['Serial_No']

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [58]:
# your code here
sum(df[['GRE_Score', 'CGPA']].duplicated())

0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [59]:
df.set_index(['GRE_Score','CGPA'],inplace=True)
df

,,Serial_No,TOEFL_Score,University_Rating,SOP,LOR,Research,Chance_of_Admit
GRE_Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
324,9.04,381,110,3,3.5,3.5,1,0.82
325,9.11,382,107,3,3.0,3.5,1,0.84
330,9.45,383,116,4,5.0,4.5,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [60]:
df.reset_index(inplace=True)
df

,GRE_Score,CGPA,Serial_No,TOEFL_Score,University_Rating,SOP,LOR,Research,Chance_of_Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [61]:
# your code here
df = pd.read_csv('../Admission_Predict.csv')
sum(df.eval('CGPA > 9 & Research == 1'))

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [62]:
# your code here
df[(df['CGPA']>9)&(df['SOP']<3.5)]
np.mean(df.query('CGPA > 9 & SOP < 3.5')['Chance of Admit '])

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [63]:
type(df['CGPA'])

pandas.core.series.Series

In [71]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (col - np.mean(col)) / np.std(col)
        
        
    
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [72]:
# your code here
df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR '])
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,14.271747,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,14.271747,1
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,27.401101,1
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,27.401101,1
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,14.271747,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,27.401101,1
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,27.401101,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,14.271747,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,3.825432,1


We will generate the decision choice at random using the code below. Please run the cell.

In [73]:
# Libraries
from random import choices

In [74]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [75]:
# your code here
df['deciding_column'] =df.lookup(df.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [76]:
# your code here
df['decision'] = np.where(df.deciding_column > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [77]:
# your code here
df.decision.value_counts()

0    298
1     87
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [79]:
# Your code here:

df.columns = (df.columns
                            .str.replace(' ', '_')
                            .str.replace('.', '')
                            .str.lower())

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [81]:
# your code here

df['adjusted_gre'] = pd.cut(np.where(df.university_rating >=4, df.gre_score + 10, df.gre_score), bins=4)

